# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import gfk_train_all, transform_gfk, gfk_compute_accuracy

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
def get_source_target_train_data(target_domain, dataset_object):
    X = None
    X_tgt = None
    for domain in dataset_object.domains:
        if target_domain == domain:
            X_tgt = dataset_object.get_all_domain_X(target_domain)
        else:
            if X is None:
                X = dataset_object.get_all_domain_X(domain)
            else:
                X_tmp = dataset_object.get_all_domain_X(domain)
                X = np.concatenate([X, X_tmp])
    
    return X, X_tgt

# Adaptación

## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/twitter/gfk/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en un diccionario separado por dominios, en donde cada dominio contiene una lista con las rutas. El diccionario es almacenado en: models/twitter/gfk/me1_models_paths.pkl


## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/twitter/gfk/best_models.pkl

# Pruebas con el dataset Twitter (2000 Dimensiones)

In [4]:
dims = 2000

In [5]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [6]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


## Creación de modelos de adaptación

In [10]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_dict = {}

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):    
    print "Cargando rutas de modelos adaptados."
    paths_dict = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parameters = {
        'dims': [int(dims/4), int(dims/2)],
        'n_subs': [10, 20, 50]
    }
    
    print "Creando modelos de adaptacion..."
    for target_domain in domains:
        print target_domain
        X , X_tgt = get_source_target_train_data(target_domain, dataset_object)
        X = X[:, :dims]
        X_tgt = X_tgt[:, :dims]
        
        folder_path = os.path.join(models_path, dataset_name, tipo)
        prefix = "me1_%d_%s_" % (dims, target_domain)
        
        new_paths = gfk_train_all(X, X_tgt, parameters, folder_path, prefix)
    
        paths_dict[target_domain] = new_paths
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_dict, models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

Cargando rutas de modelos adaptados.
Rutas cargadas en la variable 'paths_dict'


In [11]:
print "Modelos almacenados en:"
for dominio, rutas in paths_dict.items():
    print dominio
    for ruta in rutas:
        print "\t", ruta     

Modelos almacenados en:
thevoice
	models/twitter/gfk/me1_2000_thevoice_0.pkl
	models/twitter/gfk/me1_2000_thevoice_1.pkl
	models/twitter/gfk/me1_2000_thevoice_2.pkl
	models/twitter/gfk/me1_2000_thevoice_3.pkl
	models/twitter/gfk/me1_2000_thevoice_4.pkl
	models/twitter/gfk/me1_2000_thevoice_5.pkl
rio2016
	models/twitter/gfk/me1_2000_rio2016_0.pkl
	models/twitter/gfk/me1_2000_rio2016_1.pkl
	models/twitter/gfk/me1_2000_rio2016_2.pkl
	models/twitter/gfk/me1_2000_rio2016_3.pkl
	models/twitter/gfk/me1_2000_rio2016_4.pkl
	models/twitter/gfk/me1_2000_rio2016_5.pkl
general
	models/twitter/gfk/me1_2000_general_0.pkl
	models/twitter/gfk/me1_2000_general_1.pkl
	models/twitter/gfk/me1_2000_general_2.pkl
	models/twitter/gfk/me1_2000_general_3.pkl
	models/twitter/gfk/me1_2000_general_4.pkl
	models/twitter/gfk/me1_2000_general_5.pkl


In [13]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"
else:
    print "Obteniendo mejores modelos..."
    pairs = list(itertools.permutations(domains, 2))
    
    for src, tgt in pairs:
        print "src: %s - tgt: %s" % (src, tgt)
        rutas = paths_dict[tgt]
        
         # se obtienen los datos del dominio
        X_tr = labeled[src]['X_tr'][:, :dims].todense()
        y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
        
        mejor_score = 0
        best_path = None
        for ruta in rutas:
            gfk = joblib.load(ruta)
            X_deep = transform_gfk(X_tr, gfk)
            
            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > mejor_score:
                mejor_score = new_score
                best_path = ruta
        
        # se guarda la ruta del mejor modelo
        best_models[(src, tgt)] = best_path
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
src: rio2016 - tgt: thevoice
score: 0.7125
score: 0.7567
score: 0.8532
score: 0.7125
score: 0.7567
score: 0.8532
src: rio2016 - tgt: general
score: 0.7869
score: 0.8404
score: 0.8617
score: 0.7868
score: 0.8396
score: 0.8583
src: thevoice - tgt: rio2016
score: 0.7554
score: 0.7707
score: 0.8152
score: 0.7489
score: 0.7707
score: 0.8152
src: thevoice - tgt: general
score: 0.7919
score: 0.7991
score: 0.8421
score: 0.7944
score: 0.8024
score: 0.8401
src: general - tgt: rio2016
score: 0.6655
score: 0.6855
score: 0.7022
score: 0.6657
score: 0.6854
score: 0.7022
src: general - tgt: thevoice
score: 0.6658
score: 0.6861
score: 0.7046
score: 0.6667
score: 0.6851
score: 0.7046
Rutas guardadas en  models/twitter/gfk/me1_2000_best_models.pkl


In [14]:
for dominios, ruta in best_models.items():
    print "%s - %s" % (dominios, ruta)

('general', 'thevoice') - models/twitter/gfk/me1_2000_thevoice_2.pkl
('rio2016', 'general') - models/twitter/gfk/me1_2000_general_2.pkl
('general', 'rio2016') - models/twitter/gfk/me1_2000_rio2016_2.pkl
('thevoice', 'general') - models/twitter/gfk/me1_2000_general_2.pkl
('thevoice', 'rio2016') - models/twitter/gfk/me1_2000_rio2016_2.pkl
('rio2016', 'thevoice') - models/twitter/gfk/me1_2000_thevoice_2.pkl


## Pruebas de clasificación

In [15]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el par de dominios
    ruta = best_models[(src, tgt)]
    modelo = joblib.load(ruta)

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts =labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, modelo)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, modelo)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)

    # transfer loss (t)
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [16]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,r->t,rio2016,thevoice,7.362689,19.507576,12.144886
1,GFK,r->g,rio2016,general,26.998127,41.675847,14.677720
2,GFK,t->r,thevoice,rio2016,10.208711,24.274047,14.065336
3,GFK,t->g,thevoice,general,26.998127,43.242952,16.244825
4,GFK,g->r,general,rio2016,10.208711,22.368421,12.159710
5,GFK,g->t,general,thevoice,7.362689,25.426136,18.063447


In [17]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/gfk/me1_2000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (1000 Dimensiones)

In [18]:
dims = 1000

In [19]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
1000
data


In [20]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


## Creación de modelos de adaptación

In [21]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_dict = {}

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):    
    print "Cargando rutas de modelos adaptados."
    paths_dict = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parameters = {
        'dims': [int(dims/4), int(dims/2)],
        'n_subs': [10, 20, 50]
    }
    
    print "Creando modelos de adaptacion..."
    for target_domain in domains:
        print target_domain
        X , X_tgt = get_source_target_train_data(target_domain, dataset_object)
        X = X[:, :dims]
        X_tgt = X_tgt[:, :dims]
        
        folder_path = os.path.join(models_path, dataset_name, tipo)
        prefix = "me1_%d_%s_" % (dims, target_domain)
        
        new_paths = gfk_train_all(X, X_tgt, parameters, folder_path, prefix)
    
        paths_dict[target_domain] = new_paths
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_dict, models_paths)
    print "Rutas cargadas en la variable 'paths_dict'"

Creando modelos de adaptacion...
rio2016
	Entrenando modelo 0
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_0.pkl
	Entrenando modelo 1
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_1.pkl
	Entrenando modelo 2
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_2.pkl
	Entrenando modelo 3
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_3.pkl
	Entrenando modelo 4
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_4.pkl
	Entrenando modelo 5
	Guardando modelo en models/twitter/gfk/me1_1000_rio2016_5.pkl
thevoice
	Entrenando modelo 0
	Guardando modelo en models/twitter/gfk/me1_1000_thevoice_0.pkl
	Entrenando modelo 1
	Guardando modelo en models/twitter/gfk/me1_1000_thevoice_1.pkl
	Entrenando modelo 2
	Guardando modelo en models/twitter/gfk/me1_1000_thevoice_2.pkl
	Entrenando modelo 3
	Guardando modelo en models/twitter/gfk/me1_1000_thevoice_3.pkl
	Entrenando modelo 4
	Guardando modelo en models/twitter/gfk/me1_1000_thevoice_4.pkl
	Entrenando modelo 5


In [22]:
print "Modelos almacenados en:"
for dominio, rutas in paths_dict.items():
    print dominio
    for ruta in rutas:
        print "\t", ruta     

Modelos almacenados en:
thevoice
	models/twitter/gfk/me1_1000_thevoice_0.pkl
	models/twitter/gfk/me1_1000_thevoice_1.pkl
	models/twitter/gfk/me1_1000_thevoice_2.pkl
	models/twitter/gfk/me1_1000_thevoice_3.pkl
	models/twitter/gfk/me1_1000_thevoice_4.pkl
	models/twitter/gfk/me1_1000_thevoice_5.pkl
rio2016
	models/twitter/gfk/me1_1000_rio2016_0.pkl
	models/twitter/gfk/me1_1000_rio2016_1.pkl
	models/twitter/gfk/me1_1000_rio2016_2.pkl
	models/twitter/gfk/me1_1000_rio2016_3.pkl
	models/twitter/gfk/me1_1000_rio2016_4.pkl
	models/twitter/gfk/me1_1000_rio2016_5.pkl
general
	models/twitter/gfk/me1_1000_general_0.pkl
	models/twitter/gfk/me1_1000_general_1.pkl
	models/twitter/gfk/me1_1000_general_2.pkl
	models/twitter/gfk/me1_1000_general_3.pkl
	models/twitter/gfk/me1_1000_general_4.pkl
	models/twitter/gfk/me1_1000_general_5.pkl


In [23]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"
else:
    print "Obteniendo mejores modelos..."
    pairs = list(itertools.permutations(domains, 2))
    
    for src, tgt in pairs:
        print "src: %s - tgt: %s" % (src, tgt)
        rutas = paths_dict[tgt]
        
         # se obtienen los datos del dominio
        X_tr = labeled[src]['X_tr'][:, :dims].todense()
        y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
        
        mejor_score = 0
        best_path = None
        for ruta in rutas:
            gfk = joblib.load(ruta)
            X_deep = transform_gfk(X_tr, gfk)
            
            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > mejor_score:
                mejor_score = new_score
                best_path = ruta
        
        # se guarda la ruta del mejor modelo
        best_models[(src, tgt)] = best_path
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
src: rio2016 - tgt: thevoice
score: 0.7387
score: 0.8107
score: 0.8463
score: 0.7367
score: 0.8107
score: 0.8463
src: rio2016 - tgt: general
score: 0.7568
score: 0.8369
score: 0.8644
score: 0.7555
score: 0.8369
score: 0.8644
src: thevoice - tgt: rio2016
score: 0.6464
score: 0.6987
score: 0.7763
score: 0.6464
score: 0.6973
score: 0.7763
src: thevoice - tgt: general
score: 0.6297
score: 0.7234
score: 0.7924
score: 0.6292
score: 0.7243
score: 0.7924
src: general - tgt: rio2016
score: 0.5950
score: 0.6269
score: 0.6383
score: 0.5938
score: 0.6269
score: 0.6390
src: general - tgt: thevoice
score: 0.6199
score: 0.6384
score: 0.6505
score: 0.6199
score: 0.6365
score: 0.6532
Rutas guardadas en  models/twitter/gfk/me1_1000_best_models.pkl


In [24]:
for dominios, ruta in best_models.items():
    print "%s - %s" % (dominios, ruta)

('general', 'thevoice') - models/twitter/gfk/me1_1000_thevoice_5.pkl
('rio2016', 'general') - models/twitter/gfk/me1_1000_general_2.pkl
('general', 'rio2016') - models/twitter/gfk/me1_1000_rio2016_5.pkl
('thevoice', 'general') - models/twitter/gfk/me1_1000_general_2.pkl
('thevoice', 'rio2016') - models/twitter/gfk/me1_1000_rio2016_2.pkl
('rio2016', 'thevoice') - models/twitter/gfk/me1_1000_thevoice_2.pkl


## Pruebas de clasificación

In [25]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el par de dominios
    ruta = best_models[(src, tgt)]
    modelo = joblib.load(ruta)

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts =labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, modelo)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, modelo)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)

    # transfer loss (t)
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [26]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,r->t,rio2016,thevoice,19.045928,23.839962,4.794034
1,GFK,r->g,rio2016,general,30.407980,43.374691,12.966712
2,GFK,t->r,thevoice,rio2016,7.372958,21.211434,13.838475
3,GFK,t->g,thevoice,general,30.407980,44.225261,13.817281
4,GFK,g->r,general,rio2016,7.372958,21.438294,14.065336
5,GFK,g->t,general,thevoice,19.045928,26.136364,7.090436


In [27]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/gfk/me1_1000.csv
Resultados guardados.
